# Datasets

Topics:
* How are numerical data returned (and expected) in **DynamicalSystems.jl**.
* Basic `Dataset` handling.
* Neighborhoods.

---

Much of the functionality of **DynamicalSystems.jl** uses numerical data. We required a structure that would unify behavior across all functions that either return or require numerical data.

This structure is the `Dataset`.

In [1]:
# Pkg.add("DynamicalSystems")
using DynamicalSystems

INFO: Recompiling stale cache file C:\Users\datseris\.julia\lib\v0.6\ChaosTools.ji for module ChaosTools.
INFO: Recompiling stale cache file C:\Users\datseris\.julia\lib\v0.6\DynamicalSystems.ji for module DynamicalSystems.


In [2]:
x = rand(1000)
y = rand(1000)
dataset = Dataset(x,y)

2-dimensional Dataset{Float64} with 1000 points:
 0.685053   0.73252  
 0.794967   0.0889916
 0.116339   0.336318 
 0.319631   0.872275 
 0.175841   0.14791  
 0.122834   0.611689 
 0.922783   0.493622 
 0.400829   0.896963 
 0.525705   0.706049 
 0.0500803  0.0440846
 0.380059   0.832833 
 0.707066   0.469951 
 0.230633   0.540681 
 ⋮                   
 0.986044   0.267067 
 0.611678   0.0413836
 0.671049   0.350544 
 0.452864   0.751479 
 0.0665961  0.603188 
 0.721967   0.706961 
 0.482561   0.239578 
 0.957915   0.215446 
 0.131012   0.104273 
 0.669764   0.405943 
 0.150816   0.804719 
 0.36857    0.29134  


A `Dataset` is a subtype of `AbstractDataset`:

In [3]:
typeof(dataset) <: AbstractDataset

true

All subtypes of `AbstractDataset` contain their data in the field `data`:

In [4]:
dataset.data

1000-element Array{StaticArrays.SArray{Tuple{2},Float64,1,2},1}:
 [0.685053, 0.73252]   
 [0.794967, 0.0889916] 
 [0.116339, 0.336318]  
 [0.319631, 0.872275]  
 [0.175841, 0.14791]   
 [0.122834, 0.611689]  
 [0.922783, 0.493622]  
 [0.400829, 0.896963]  
 [0.525705, 0.706049]  
 [0.0500803, 0.0440846]
 [0.380059, 0.832833]  
 [0.707066, 0.469951]  
 [0.230633, 0.540681]  
 ⋮                     
 [0.986044, 0.267067]  
 [0.611678, 0.0413836] 
 [0.671049, 0.350544]  
 [0.452864, 0.751479]  
 [0.0665961, 0.603188] 
 [0.721967, 0.706961]  
 [0.482561, 0.239578]  
 [0.957915, 0.215446]  
 [0.131012, 0.104273]  
 [0.669764, 0.405943]  
 [0.150816, 0.804719]  
 [0.36857, 0.29134]    

In [5]:
typeof(dataset.data)

Array{StaticArrays.SArray{Tuple{2},Float64,1,2},1}

* This means that the data are contained in a `Vector` of `SVector`s (which are statically sized vectors).

We chose a vector of `SVectors` as the internal representation of the data because it gives big performance gains in many functions of the library.

# Creating a Dataset

Most functions of **DynamicalSystems.jl** that return numerical data, return it in the form of a `Dataset`.

If you already have the data though, there are many ways to create a dataset:

In [6]:
# From matrix where each column is one variable:
m = rand(1000, 5)
data = Dataset(m)

5-dimensional Dataset{Float64} with 1000 points:
 0.938389   0.211422   0.226546    0.763744   0.897615 
 0.549869   0.247598   0.964724    0.808817   0.666845 
 0.0948477  0.0259562  0.903702    0.0762062  0.670173 
 0.659321   0.682475   0.468257    0.101155   0.172539 
 0.551967   0.133988   0.968181    0.217272   0.763818 
 0.616987   0.390742   0.627401    0.336416   0.326163 
 0.252229   0.457863   0.37837     0.362139   0.634866 
 0.625016   0.90596    0.23407     0.429219   0.0805099
 0.768675   0.627347   0.322967    0.258017   0.803529 
 0.183717   0.265727   0.723414    0.732449   0.799991 
 0.500832   0.503597   0.785832    0.48966    0.0521491
 0.700686   0.14947    0.164091    0.851882   0.208668 
 0.105927   0.87894    0.173601    0.890038   0.467465 
 ⋮                                                     
 0.154146   0.720938   0.747165    0.284099   0.0420155
 0.977539   0.159134   0.0332846   0.65252    0.187167 
 0.341352   0.802107   0.00794109  0.660082   0.768523 

In [7]:
# use re-interpret if each *row* is one variable:
m = transpose(m)
data2 = reinterpret(Dataset, m)

5-dimensional Dataset{Float64} with 1000 points:
 0.938389   0.211422   0.226546    0.763744   0.897615 
 0.549869   0.247598   0.964724    0.808817   0.666845 
 0.0948477  0.0259562  0.903702    0.0762062  0.670173 
 0.659321   0.682475   0.468257    0.101155   0.172539 
 0.551967   0.133988   0.968181    0.217272   0.763818 
 0.616987   0.390742   0.627401    0.336416   0.326163 
 0.252229   0.457863   0.37837     0.362139   0.634866 
 0.625016   0.90596    0.23407     0.429219   0.0805099
 0.768675   0.627347   0.322967    0.258017   0.803529 
 0.183717   0.265727   0.723414    0.732449   0.799991 
 0.500832   0.503597   0.785832    0.48966    0.0521491
 0.700686   0.14947    0.164091    0.851882   0.208668 
 0.105927   0.87894    0.173601    0.890038   0.467465 
 ⋮                                                     
 0.154146   0.720938   0.747165    0.284099   0.0420155
 0.977539   0.159134   0.0332846   0.65252    0.187167 
 0.341352   0.802107   0.00794109  0.660082   0.768523 

In [8]:
data2 == data

true

In [9]:
# From individual columns
x, y = rand(1000), rand(1000)
data = Dataset(x, y)

# All points of a dataset must be equally sized:
z = rand(100)
Dataset(x, z)

LoadError: [91mBoundsError: attempt to access 100-element Array{Float64,1} at index [101][39m

# Handling a Dataset

** Subtypes of `AbstractDataset` behave like some `Matrix` where each *row* is a *point* of the dataset, while each *column* is *one dynamic variable*. **

* The reason we choose this approach is because traditionally this is how scientific data are recorded, exported and passed around.

In [10]:
# Create some random dataset:
dataset = Dataset(rand(10000, 3));
typeof(dataset)

DynamicalSystemsBase.Dataset{3,Float64}

In [11]:
dataset[:, 2] # this is the second variable series

10000-element Array{Float64,1}:
 0.228718
 0.888424
 0.977115
 0.155934
 0.88932 
 0.145428
 0.430117
 0.90077 
 0.601241
 0.296718
 0.359361
 0.681379
 0.504481
 ⋮       
 0.178406
 0.680972
 0.688331
 0.385725
 0.588239
 0.498226
 0.423515
 0.956837
 0.995455
 0.193505
 0.736879
 0.159393

In [12]:
dataset[1, :] # this is the first datapoint (3-dimensional)

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.532608
 0.228718
 0.848861

In [13]:
dataset[1] # acessing with a single index is like accessing a vector of points

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.532608
 0.228718
 0.848861

In [14]:
dataset[5, 2] # value of the second variable, at the 5th timepoint

0.8893197684782828

In [15]:
# get columns
x, y = columns(dataset)
mean(x), mean(y)

(0.4997359628293707, 0.5005101728772056)

In [16]:
# iteration:
f = 0.0
for point ∈ dataset  # same as `point ∈ dataset` , do \in<tab> for ∈
    f += mean(point)
end
f

5015.252874108764

In [17]:
# minima, maxima
mini = minima(dataset)
maxi = maxima(dataset)
mini, maxi = minmaxima(dataset)

([0.000148504, 0.000173894, 7.52062e-5], [0.999936, 0.999983, 0.999981])

In [18]:
# Function minmaxima is faster than using the two individual ones
using BenchmarkTools
@btime maxima($dataset)
@btime minima($dataset)
@btime minmaxima($dataset);

  26.685 μs (1 allocation: 112 bytes)
  26.685 μs (1 allocation: 112 bytes)
  45.981 μs (2 allocations: 224 bytes)


# I/O
Input/output functionality for an `AbstractDataset` is already achieved using base Julia:
1. `writedlm` for writting
2. `readdlm` for reading.

To write and read a dataset, simply use the `data` field:

In [19]:
dataset = Dataset(rand(1000,3))

# I will write and read using delimiter ','
writedlm("data.txt", dataset.data, ',')

# Load the data from text:
data = Dataset(readdlm("data.txt", ',', Float64))
# Don't forget to convert the matrix to a Dataset when reading!

3-dimensional Dataset{Float64} with 1000 points:
 0.156433   0.996693    0.760031  
 0.449242   0.830958    0.700282  
 0.228454   0.46779     0.702444  
 0.71223    0.535813    0.861171  
 0.422346   0.132154    0.111887  
 0.559868   0.151468    0.583483  
 0.406699   0.00833475  0.567344  
 0.452598   0.121519    0.982545  
 0.437886   0.67973     0.839332  
 0.684247   0.344526    0.0967143 
 0.260073   0.583528    0.547732  
 0.912595   0.723635    0.478863  
 0.0173768  0.815204    0.902385  
 ⋮                                
 0.139292   0.897499    0.0726134 
 0.921404   0.254743    0.656177  
 0.966753   0.643486    0.488232  
 0.191791   0.374992    0.291254  
 0.256613   0.288416    0.00841836
 0.282451   0.181273    0.981363  
 0.697728   0.0568779   0.683246  
 0.376258   0.760088    0.322551  
 0.458258   0.645002    0.952668  
 0.309075   0.351741    0.906157  
 0.774603   0.818155    0.625893  
 0.585434   0.0391121   0.514847  


In [20]:
# delete the dummy file we created:
rm("data.txt")
isfile("data.txt")

false

# Neighborhoods 

A "neighborhood" is a collection of points that is near a given point. `Dataset`s interface the module [`NearestNeighbors`](https://github.com/KristofferC/NearestNeighbors.jl) in order to find this neighborhood.

We use the function `neighborhood`. The call singature is:
```julia
neighborhood(point, tree, ntype)
```


`point` is simply the query point. `tree` is the structure required by [`NearestNeighbors`](https://github.com/KristofferC/NearestNeighbors.jl), and is obtained simply by:

In [21]:
tree = KDTree(dataset)

NearestNeighbors.KDTree{StaticArrays.SArray{Tuple{3},Float64,1,3},Distances.Euclidean,Float64}
  Number of points: 1000
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

The third argument to `neighborhood` is the *type* of the neighborhood. 

* There are two types of neighborhoods!

The first one is defined as the `k` nearest points to a given point. It is represented in code by:

In [22]:
mybuddies = FixedMassNeighborhood(3) # for experts: does a knn search

DynamicalSystemsBase.FixedMassNeighborhood(3)

In [23]:
point = ones(3)
n = neighborhood(point, tree, mybuddies)

3-element Array{Int64,1}:
 374
 245
 169

Notice that the `neighborhood` function does not return the points themselves, but rather the indices of the points in the original data:

In [24]:
println("neighborhood of $(point) is:")

for i in n
    println(dataset[i])
end

neighborhood of [1.0, 1.0, 1.0] is:
[0.894917, 0.911732, 0.980697]
[0.993621, 0.987839, 0.881758]
[0.877541, 0.937721, 0.986534]


---

The second type of neighborhood is all the points that are within some given distance `ε` from a point.

In code, we represent this as:

In [25]:
where_u_at = FixedSizeNeighborhood(0.001) # for experts: does an inrange search

DynamicalSystemsBase.FixedSizeNeighborhood(0.001)

In [26]:
n2 = neighborhood(point, tree, where_u_at)

0-element Array{Int64,1}

In [27]:
plz_come_closer = FixedSizeNeighborhood(0.2)
n2 = neighborhood(point, tree, plz_come_closer)

5-element Array{Int64,1}:
 350
 169
 374
 245
 669

In [28]:
println("neighborhood of $(point) is:")

for i in n2
    println(dataset[i])
end

neighborhood of [1.0, 1.0, 1.0] is:
[0.867781, 0.959831, 0.958156]
[0.877541, 0.937721, 0.986534]
[0.894917, 0.911732, 0.980697]
[0.993621, 0.987839, 0.881758]
[0.987789, 0.810901, 0.988053]


Okay, so points that have distance < ε are accepted as a neighborhood.

What is the "distance" though? When defining a `tree`, you can optionally give a distance function. By default this is the Euclidean distance, but others also work:

In [29]:
using Distances # to get distance functions
funky_tree = KDTree(dataset, Chebyshev())

NearestNeighbors.KDTree{StaticArrays.SArray{Tuple{3},Float64,1,3},Distances.Chebyshev,Float64}
  Number of points: 1000
  Dimensions: 3
  Metric: Distances.Chebyshev()
  Reordered: true

In [30]:
n3 = neighborhood(point, funky_tree, plz_come_closer)

10-element Array{Int64,1}:
 824
 350
 169
 642
 144
 275
 374
 245
 686
 669

# Excluding temporal neighbors

Before moving on, let's see one last thing:

In [31]:
# the point I want the neighborhood is now part of my dataset:
point = dataset[end]

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.585434 
 0.0391121
 0.514847 

In [32]:
# Let's calculate again the two neighborhoods

tree = KDTree(dataset)

# Find suuuuuuuper close neighbors:
ε = 0.000001
where_u_at = FixedSizeNeighborhood(ε)
n2 = neighborhood(point, tree, where_u_at)

# Find the nearest neighbor:
my_best_friend = FixedMassNeighborhood(1)
n3 = neighborhood(point, tree, my_best_friend)

println(n2)
println(n3)

[1000]
[1000]


In [33]:
length(dataset) == n2[1] == n3[1]

true

*Apparently my best friend is myself. Who would have thought...*

**What is happening here is that the `neighborhood` also counted the point itself, since it is also part of the dataset.**

* Almost always this behavior needs to be avoided. For this reason, there is a second method for `neighborhood`:

```julia
neighborhood(point, tree, ntype, idx::Int, w::Int = 1)
```

In this case, `idx` is the index of the point in the original data. `w` stands for the Theiler window (positive integer).

Only points that have index
`abs(i - idx) ≥ w` are returned as a neighborhood, to exclude close temporal neighbors.

* The default `w=1` is the case of exluding the `point` itself.

Let's revisit the last example (using the default value of `w = 1`):

In [34]:
point = dataset[end]
idx = length(dataset)

n2 = neighborhood(point, tree, where_u_at, idx)
n3 = neighborhood(point, tree, my_best_friend, idx)

println(n2)
println(n3)

Int64[]
[216]


As you can see, there isn't *any* neighbor of `point` with distance `< 0.000001` in this dataset, but there is always a nearest neighbor:

In [35]:
println(dataset[n3[1]], " is the nearest neighbor of ", point)

[0.588396, 0.100388, 0.524337] is the nearest neighbor of [0.585434, 0.0391121, 0.514847]


# Documentation Strings

In [36]:
?Dataset

search: Dataset AbstractDataset DynamicalSystemsBase @dateformat_str



```
Dataset{D, T} <: AbstractDataset{D,T}
```

A dedicated interface for datasets, i.e. vectors of vectors. It contains *equally-sized datapoints* of length `D`, represented by `SVector{D, T}`.

It can be used exactly like a matrix that has each of the columns be the timeseries of each of the dynamic variables. [`trajectory`](@ref) always returns a `Dataset`. For example,

```julia
ds = Systems.towel()
data = trajectory(ds, 1000) #this returns a dataset
data[:, 2] # this is the second variable timeseries
data[1] == data[1, :] # this is the first datapoint (D-dimensional)
data[5, 3] # value of the third variable, at the 5th timepoint
```

Use `Matrix(dataset)` or `reinterpret(Matrix, dataset)` and `Dataset(matrix)` or `reinterpret(Dataset, matrix)` to convert. The `reinterpret` methods are cheaper but assume that each variable/timeseries is a *row* and not column of the `matrix`.

If you have various timeseries vectors `x, y, z, ...` pass them like `Dataset(x, y, z, ...)`. You can use `columns(dataset)` to obtain the reverse, i.e. all columns of the dataset in a tuple.


In [37]:
?neighborhood

search: neighborhood AbstractNeighborhood FixedSizeNeighborhood



```
neighborhood(point, tree, ntype)
neighborhood(point, tree, ntype, n::Int, w::Int = 1)
```

Return a vector of indices which are the neighborhood of `point` in some `data`, where the `tree` was created using `tree = KDTree(data [, metric])`. The `ntype` is the type of neighborhood and can be any subtype of [`AbstractNeighborhood`](@ref).

Use the second method when the `point` belongs in the data, i.e. `point = data[n]`. Then `w` stands for the Theiler window (positive integer). Only points that have index `abs(i - n) ≥ w` are returned as a neighborhood, to exclude close temporal neighbors. The default `w=1` is the case of excluding the `point` itself.

## References

`neighborhood` simply interfaces the functions `knn` and `inrange` from [NearestNeighbors.jl](https://github.com/KristofferC/NearestNeighbors.jl) by using the argument `ntype`.


In [38]:
?AbstractNeighborhood

search: AbstractNeighborhood



```
AbstractNeighborhood
```

Supertype of methods for deciding the neighborhood of points for a given point.

Concrete subtypes:

  * `FixedMassNeighborhood(K::Int)` : The neighborhood of a point consists of the `K` nearest neighbors of the point.
  * `FixedSizeNeighborhood(ε::Real)` : The neighborhood of a point consists of all neighbors that have distance < `ε` from the point.

See [`neighborhood`](@ref) for more.
